# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева, НИУ ВШЭ*

## Практикум 7.1. Управление браузером с Selenium: динамические таблицы

Импортируем необходимые модули и коллекции методов:

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep

from selenium import webdriver as wd
from selenium.webdriver.common.by import By

Сохраняем ссылку на страницу со статистикой по футбольным матчам в переменную `url` (специально выбран довольно специфический ресурс, с кучей лишних окон, картинок и рекламы, которые потенциально могут мешать):

In [2]:
url = "https://1xbet.whoscored.com/Statistics"
print(url)

https://1xbet.whoscored.com/Statistics


Открываем новое окно браузера через Selenium и переходим на страницу по ссылке `url`:

In [3]:
br = wd.Chrome()
br.get(url)

### Задача 1

Извлеките исходный код страницы и найдите в нём первую таблицу с результатами. Преобразуйте её в полноценный датафрейм pandas.

In [4]:
html = br.page_source
soup = BeautifulSoup(html)
tab = soup.find("table", 
                {"id" : "top-team-stats-summary-grid"})
df = pd.read_html(str(tab))[0]

In [5]:
# все отлично, но мы видим только один фрагмент таблицы
# так как для дальнейших результатов ее надо листать

print(df.shape)
print(df.head())

(20, 9)
                     Team      Tournament  Goals  Shots pg  Discipline  \
0        1. Bayern Munich      Bundesliga     78      20.2         422   
1     2. Bayer Leverkusen      Bundesliga     66      17.7         470   
2  3. Paris Saint-Germain         Ligue 1     62      15.0         372   
3      4. Manchester City  Premier League     63      18.0         462   
4          5. Real Madrid          LaLiga     64      16.4         524   

   Possession%  Pass%  AerialsWon  Rating  
0         62.0   88.6        12.5    7.01  
1         63.1   88.9        10.4    6.97  
2         65.6   89.8         8.0    6.95  
3         65.3   90.1         9.2    6.93  
4         59.6   90.1         8.7    6.89  


### Задача 2

Напишите функцию `get_table()`, которая принимает на вход объект `br` и возвращает датафрейм с результатами из первой таблицы на странице. 

In [6]:
from time import sleep

In [7]:
def get_table(br):
    sleep(2)
    html = br.page_source
    soup = BeautifulSoup(html)
    tab = soup.find("table", 
                {"id" : "top-team-stats-summary-grid"})
    df = pd.read_html(str(tab))[0]
    return df

### Задача 3

Напишите программу, которая пролистывает все страницы в первой таблице и сохраняет все данные из нее в датафрейм pandas.

**NB.** Перед запуском кода убедитесь, что окно браузера открыто полностью (по крайней мере на всю ширину экрана, иначе некоторые элементы меню будут перекрывать нужную кнопку для перелистывания.

In [8]:
# while True: исполняем код, пока не столкнемся с выходом из цикла break
# а столкнемся мы с ним тогда, когда не будет выполняться условие в if
# условие проверяет, правда ли у элемента button_next в атрибуте class
# указано значение option  clickable 
# –> кнопка для листания кликабельна, дальше есть, куда листать

tables = []
while True:
    res = get_table(br)
    tables.append(res)
    button_next = br.find_element(By.ID, "next")
    if button_next.get_dom_attribute("class") == "option  clickable ":
        button_next.click()
        sleep(3)
    else:
        break

In [10]:
fin = pd.concat(tables)
print(fin.head())
print(fin.shape) # все 96 строк

                     Team      Tournament  Goals  Shots pg  Discipline  \
0        1. Bayern Munich      Bundesliga     78      20.2         422   
1     2. Bayer Leverkusen      Bundesliga     66      17.7         470   
2  3. Paris Saint-Germain         Ligue 1     62      15.0         372   
3      4. Manchester City  Premier League     63      18.0         462   
4          5. Real Madrid          LaLiga     64      16.4         524   

   Possession%  Pass%  AerialsWon  Rating  
0         62.0   88.6        12.5    7.01  
1         63.1   88.9        10.4    6.97  
2         65.6   89.8         8.0    6.95  
3         65.3   90.1         9.2    6.93  
4         59.6   90.1         8.7    6.89  
(96, 9)
